In [1]:
import os
import json
import glob
import shutil
from math import dist
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.font_manager import FontProperties

In [2]:
class CDF():
    def __init__(self, src):
        self.src = src
        self.ear_types = ["free","attached"]
        self.degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg',
                        '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']
        self.acupoints_name = ["HX6(肛門)", "HX3(尿道)", "HX1(耳中)", "TF4(神門)", "AH9(腰柢椎)", "AH11(胸椎)", "CO5(十二指腸)", 
                               "CO4(胃)", "CO3(賁門)", "CO14(肺)", "CO15(心)", "CO18(內分泌)", "HX2(輪2)", "AT2(緣中)", 
                               "LO8(扁桃體)", "LO1(牙)" , "LO2(舌)", "LO3(頷)", "LO4(垂前)", "LO5(眼)", "LO6(內耳)"]
        self.cm = {"cather":4.72, "daniel":4.74, "dominic":3.9, "jack":3.91, "jakaria":4.20, "jimmy":4.60, "vicky":4.69, "wayne":4.75,
                   "gaby":4.05, "matt":5.0, "toby":4.26, "wendy":4.14, "willy":4.51}
    def read_error_csv(self, fpath1):
        error_np = pd.read_csv(fpath1, index_col = 0).to_numpy()
        return error_np
    def cal_CDF(self):
        final = {}
        for ear_type in self.ear_types:
            final[ear_type] ={}
            total = []
            names = os.listdir(os.path.join("..", self.src, ear_type, "result"))
            for name in names:
                for deg in self.degrees:
                    error_np = self.read_error_csv(os.path.join("..", self.src, ear_type, "result", name, "error", deg, "error.csv"))
                    interval = len(error_np) / 3
                    index = [0, int(interval), int(interval*2), len(error_np)-1]
                    for i in index:
                        total.append(error_np[i])
            total = np.array(total)
            total = total.reshape(-1)
            sorted_total = np.sort(total*10)
            cumulative_prob_total = np.arange(1, len(total) + 1) / len(total)
            final[ear_type]["sorted_error"] = sorted_total
            final[ear_type]["cumulative_prob"] = cumulative_prob_total
        return final     

In [3]:
class CDF_Draw():
    def __init__(self, CDF_0, CDF_1):
        self.CDF_0 = CDF_0
        self.CDF_1 = CDF_1
        
    def Draw_one_CDF(self, title, label):
        for ear_type in self.CDF_0.ear_types:
            plt.plot(self.CDF_0.cal_CDF()[ear_type]["sorted_error"], self.CDF_0.cal_CDF()[ear_type]["cumulative_prob"], label="The result of {label_}".format(label_ = label))
            plt.xlim(0, 10)
            plt.ylim(0, 1.1)
            plt.xlabel("mm")
            plt.ylabel("Probability")
            plt.title(title)
            plt.legend()
            plt.show()
#             if not os.path.isdir(os.path.join("../mmpose/k_fold", "CDF", ear_type)):
#                 os.makedirs(os.path.join("../mmpose/k_fold", "CDF", ear_type))
#             plt.savefig(os.path.join("../mmpose/k_fold", "CDF", ear_type, 'CDF.png').format(index = 0))
            
    def Draw_two_CDF(self, title, label_0, label_1):
        for ear_type in self.CDF_0.ear_types:
            plt.plot(self.CDF_0.cal_CDF()[ear_type]["sorted_error"], self.CDF_0.cal_CDF()[ear_type]["cumulative_prob"], label="The result of {label_}".format(label_ = label_0))
            plt.plot(self.CDF_1.cal_CDF()[ear_type]["sorted_error"], self.CDF_1.cal_CDF()[ear_type]["cumulative_prob"], label="The result of {label_}".format(label_ = label_1))
            plt.xlim(0, 10)
            plt.ylim(0, 1.1)
            plt.xlabel("mm")
            plt.ylabel("Probability")
            plt.title(title)
            plt.legend()
            if not os.path.isdir(os.path.join("..", "figure", "diff_inpainting_way", "CDF")):
                os.makedirs(os.path.join("..", "figure", "diff_inpainting_way", "CDF"))
            plt.savefig(os.path.join("..", "figure", "diff_inpainting_way", "CDF", 'CDF_{ear_type}.png'.format(ear_type = ear_type)))
            plt.show()
            

In [5]:
A = CDF("MAT_inpainting")
B = CDF("traditional_inpainting")
C = CDF_Draw(A, B)
C.Draw_one_CDF("123","321")
C.Draw_two_CDF("CDF Comparison of Different Inpainting Methods","MAT","OPENCV")